In [ ]:
!pip install -U -q PyDrive

In [ ]:
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.1 MB/s eta 0:00:00


In [ ]:
import transformers

print(transformers.__version__)

4.28.0


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("question_answering_notebook", framework="pytorch")

In [ ]:
model_checkpoint = "xlnet-base-cased"
batch_size = 16

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
datasets = load_dataset('squad')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 384 
doc_stride = 128

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForQuestionAnsweringSimple: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-uylwxdix
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-uylwxdix
  Resolved https://github.com/huggingface/accelerate to commit dcde1e93d09abea02a8e7f4a07a2c5734b87b60e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=222965 sha256=628e89d143ac0f4ba77a8284a93455c18bc0cb8923ab4c472e9487a1796a23cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-i3ydckz5/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
from datasets import Dataset

In [ ]:
ind=int(0.2*(tokenized_datasets["train"].num_rows))
tokenized_dataset_train = Dataset.from_dict(tokenized_datasets["train"][:ind])

In [ ]:
ind=int(0.2*(tokenized_datasets["validation"].num_rows))
tokenized_dataset_val = Dataset.from_dict(tokenized_datasets["validation"][:ind])

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.335700,0.316447


Epoch,Training Loss,Validation Loss
1,0.335700,0.316447
2,0.199800,0.326509


TrainOutput(global_step=2222, training_loss=0.34803036845127383, metrics={'train_runtime': 5416.1001, 'train_samples_per_second': 6.561, 'train_steps_per_second': 0.41, 'total_flos': 7543849924721664.0, 'train_loss': 0.34803036845127383, 'epoch': 2.0})

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/XLNet_QA_clean")

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
ind=int(0.2*(datasets["validation"].num_rows))
datasets_valid = Dataset.from_dict(datasets["validation"][:ind])

In [ ]:
validation_features = datasets_valid.map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets_valid.column_names
)

Map:   0%|          | 0/2114 [00:00<?, ? examples/s]

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
max_answer_length = 30

In [ ]:
import collections

examples = datasets_valid
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
          # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(datasets_valid, validation_features, raw_predictions.predictions)

Post-processing 2114 example predictions split into 2142 features.


  0%|          | 0/2114 [00:00<?, ?it/s]

In [ ]:
metric = load_metric("squad")

<ipython-input-55-7143d13414c2>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")


In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets_valid]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 7.426679280983917, 'f1': 8.560132156633506}

### **GROUND TRUTH vs MODEL PREDICITIONS**

In [ ]:
[[['Kelvin Benjamin', 'Kelvin Benjamin', 'Benjamin'], 'longtime'],
 [['six', 'six', 'six'], 'six'],
 [['45', '45', '45'], '45'],
 [['10', '10', '10'], '10'],
 [['27', '27', '27'], '27'],
 [['Greg Olsen', 'Greg Olsen', 'Olsen'], 'Greg Olsen'],
 [['45', '45', '45'], '45'],
 [['99.4', '99.4', '99.4.'], '99.4'],
 [['77 passes', '77', '77'], '77'],
 [['receivers', 'receivers', 'receivers'], 'six Pro Bowl selections.'],
 [['Jonathan Stewart', 'Jonathan Stewart', 'Jonathan Stewart'], 'Cam Newton'],
 [['six', 'six', 'six'], 'six'],
 [['Cam Newton', 'Cam Newton', 'Cam Newton'], 'Cam Newton'],
 [['3,837', '3,837', '3,837'], '3,837 yards and rushing for 636'],
 [['45', '45', '45'], '45'],
 [['six', 'six', 'six'], '45'],
 [['500', '500', '500'], '500 points'],
 [['3,837', '3,837', '3,837'], '636'],
 [['45', '45', '45'], '45'],
 [['99.4.', '99.4', '99.4.'], '99.4'],
 [['39', '39', '39'], '39 receptions for 485'],
 [['308', '308', '308'], '308'],
 [['136', '136', '136'], '136'],
 [['118', '118', '118'], '11'],
 [['four', 'four', 'four'], '24'],
 [['Kawann Short', 'Kawann Short', 'Short'], 'Kawann Short'],
 [['24', '24', '24'], '24'],
 [['Kawann Short', 'Kawann Short', 'Short'], 'Kawann Short'],
 [['four', 'four', 'four'], 'four'],
 [['four', '5½', 'four'], '11'],
 [['Kurt Coleman', 'Kurt Coleman', 'Coleman'], 'Kawann Short'],
 [['24', '24', '24'], '24'],
 [['Kony Ealy', 'Kony Ealy'], 'Kony Ealy'],
 [['Luke Kuechly.', 'Luke Kuechly', 'Short'], 'Kawann Short'],
 [['two.', 'two', '24'], '24'],
 [['Gary Kubiak', 'Gary Kubiak', 'Kubiak'], 'Gary Kubiak'],
 [['Brock Osweiler', 'Brock Osweiler', 'Osweiler'], 'Brock Osweiler'],
 [['Indianapolis Colts', 'the Indianapolis Colts', 'Colts'],
  'Indianapolis Colts'],
 [['San Diego Chargers', 'San Diego Chargers', 'Chargers'], 'Peyton Manning'],
 [['Wade Phillips', 'Wade Phillips', 'Phillips'], 'Wade Phillips'],
 [['four', 'four', 'four'], 'four'],
 [['Gary Kubiak', 'Gary Kubiak', 'Kubiak'], 'Gary Kubiak'],
 [['Indianapolis Colts', 'the Indianapolis Colts', '1998'],
  'Indianapolis Colts'],
 [['39', '39', '39'], '39'],
 [['plantar fasciitis', 'a partial tear of the plantar fasciitis'],
  'plantar fasciitis'],
 [['Gary Kubiak', 'Gary Kubiak', 'Kubiak'], 'Gary Kubiak'],
 [['Peyton Manning', 'Peyton Manning', 'Manning'], 'Peyton Manning'],
 [['a plantar fasciitis injury',
   'a plantar fasciitis injury',
   'plantar fasciitis'],
  'plantar fasciitis'],
 [['39', '39', '39'], '39'],
 [['four', '4', '4'], 'the divisional round'],
 [['John Fox', 'John Fox', 'Fox'], 'Gary Kubiak'],
 [['Peyton Manning', 'Peyton Manning', 'Manning'],
  "Following their loss in the divisional round of the previous season's playoffs, the Denver Broncos underwent numerous coaching changes, including a mutual parting with head coach John Fox (who had won four divisional championships in his four years as Broncos head coach), and the hiring of Gary Kubiak as the new head coach. Under Kubiak, the Broncos planned to install a run-oriented offense with zone blocking to blend in with quarterback Peyton Manning's shotgun passing skills, but struggled with numerous changes and injuries to the offensive line, as well as Manning having his worst statistical season since his rookie year with the Indianapolis Colts in 1998, due to a plantar fasciitis injury in his heel that he had suffered since the summer, and the simple fact that Manning was getting old, as he turned 39 in the 2015 off-season. Although the team had a 7–0 start, Manning led the NFL in interceptions. In week 10, Manning suffered a partial tear of the plantar fasciitis in his left foot. He set the NFL's all-time record for career passing yards in this game, but"],
 [['Gary Kubiak', 'Gary Kubiak', 'Kubiak'], 'Gary Kubiak'],
 [['left foot.', 'left', 'left'], 'plantar fasciitis injury in his heel'],
 [['Wade Phillips', 'Wade Phillips', 'Phillips'], 'Wade Phillips'],
 [['67.9', '67.9', '67.9'], '67.9'],
 [['17', '17', '17'], '17 interceptions'],
 [['Demaryius Thomas', 'Demaryius Thomas', 'Thomas'], 'Manning'],
 [['C. J. Anderson', 'C. J. Anderson', 'Anderson'], 'Manning'],
 [['10', '10', '10'], '1,967'],
 [['67.9', '67.9', '67.9'], '67.9'],
 [['2,249', '2,249', '2,249'], '2,249'],
 [['nine', '17', 'career-low 67.9 passer rating,'], '17 interceptions'],
 [['Demaryius Thomas', 'Demaryius Thomas', 'Thomas'], 'Manning'],
 [['receiver', 'receiver', 'Thomas'], 'Veteran'],
 [['67.9', '67.9', '67.9'], '67.9'],
 [['17', '17', '17'], '17'],
 [['Demaryius Thomas', 'Demaryius Thomas', 'Thomas'], 'Demaryius'],
 [['5', 'five', 'five'], '17 interceptions.'],
 [['67.9', '67.9', '67.9'], '67.9'],
 [['17', '17', '17'], '17'],
 [['Emmanuel Sanders', 'Emmanuel Sanders', 'Sanders'], 'Manning'],
 [['C. J. Anderson', 'C. J. Anderson', 'Anderson'], 'Manning'],
 [['4.7', '4.7', '4.7'], '67.9 passer rating, throwing for 2,249'],
 [['Justin Tucker', 'Justin Tucker', 'Tucker'], 'Concerns'],
 [['Bermuda 419', 'Bermuda 419', 'hybrid Bermuda 419'], 'high enough quality'],
 [['Ed Mangan', 'Ed Mangan', 'Mangan'], "Levi's Stadium"],
 [['Baltimore Ravens', 'Baltimore Ravens', 'Ravens'], "Levi's Stadium"],
 [['kicker', 'kicker', 'kicker'], "Levi's Stadium"],
 [['Justin Tucker', 'Justin Tucker', 'Tucker'], 'Concerns'],
 [['kicker', 'kicker', 'kicker'], "Levi's Stadium"],
 [['hybrid Bermuda 419 turf', 'Bermuda 419 turf', 'hybrid Bermuda 419'],
  'Concerns'],
 [['Justin Tucker', 'Justin Tucker', 'Tucker'], "Levi's Stadium"],
 [['a new playing surface', 're-sodded', 're-sodded'], 'high enough quality'],
 [['a hybrid Bermuda 419 turf.',
   'Bermuda 419 turf',
   'hybrid Bermuda 419 turf'],
  'Concerns'],
 [['their cleats', 'cleats', 'cleats'], "Levi's Stadium"],
 [['Justin Tucker', 'Tucker'], 'Super Bowl'],
 [['natural grass', 'natural grass', 'grass'], "Levi's Stadium"],
 [['Broncos', 'the Broncos', 'Broncos'], 'Elway'],
 [['34–19', '34–19', '34–19'], '34–19'],
 [['Atlanta Falcons', 'the Atlanta Falcons', 'Falcons'], 'Atlanta Falcons'],
 [['white', 'white', 'white'], 'road white jerseys'],
 [['Super Bowl XXXIII', 'Super Bowl XXXIII', 'Super Bowl XXXIII'],
  'Super Bowl XXXIII'],
 [['Super Bowl XXXIII', 'Super Bowl XXXIII', 'Super Bowl XXXIII'],
  'Super Bowl XXXIII'],
 [['34–19', '34–19', '34–19'], '34–19'],
 [['Atlanta Falcons', 'the Atlanta Falcons', 'Falcons'], 'Atlanta Falcons'],
 [['white', 'matching white jerseys', 'white'], 'road white'],
 [['road white jerseys', 'matching white', 'white'], 'road white jerseys'],
 [['Pittsburgh Steelers', 'the Pittsburgh Steelers', 'Steelers'],
  'AFC and NFC teams, the Broncos elected to wear their road white jerseys with matching white pants'],
 [['Super Bowl XXXIII', 'Super Bowl XXXIII', 'XXXIII'], 'Super Bowl XXXIII'],
 [['blue', 'blue', 'white'], 'road white'],
 [['orange', 'orange', 'orange'], 'road white jerseys'],
 [['black jerseys with silver pants.',
   'black jerseys with silver pants',
   'black'],
  'road white jerseys'],
 [['$5,000,000', '$5,000,000', '$5,000,000,'], '$5,000,000'],
 [['Anheuser-Busch InBev', 'Anheuser-Busch InBev', 'Anheuser-Busch InBev'],
  'CBS'],
 [['Doritos', 'Doritos', 'Doritos'], 'CBS'],
 [['20th', 'the 20th', '20th'],
  'CBS set the base rate for a 30-second advertisement at $5,000,000'],
 [['$5,000,000', '$5,000,000', '$5,000,000'], '$5,000,000'],
 [['Anheuser-Busch InBev', 'Anheuser-Busch InBev', 'Anheuser-Busch InBev'],
  'CBS'],
 [['Doritos', 'Doritos', 'Doritos'], 'CBS'],
 [['Nintendo', 'Nintendo', 'Nintendo'],
  'CBS set the base rate for a 30-second advertisement at $5,000,000'],
 [['The Pokémon Company', 'The Pokémon Company', 'The Pokémon Company'],
  'CBS set the base rate for a 30-second advertisement at $5,000,000, a record high price for a Super Bowl ad.'],
 [['Anheuser-Busch InBev', 'Anheuser-Busch InBev', 'Anheuser-Busch InBev'],
  'CBS'],
 [['Doritos', 'Doritos', 'Doritos'], 'CBS'],
 [['Anheuser-Busch InBev', 'Anheuser-Busch InBev', 'Anheuser-Busch InBev'],
  'CBS'],
 [['Doritos', 'Doritos', 'Doritos'], 'CBS'],
 [['Crash the Super Bowl', 'Crash the Super Bowl', 'Crash the Super Bowl'],
  '$5,000,000'],
 [['KRFX', 'KRFX', 'KRFX'], 'KOA (850 AM) and KRFX'],
 [['Dave Logan', 'KOA (850 AM) and KRFX (103.5 FM)', 'Dave Logan'],
  'Dave Logan'],
 [['1110 AM', '1110 AM', '(1110 AM)'], '850'],
 [['Chester, South Carolina', 'Chester', 'Chester, South Carolina'], 'Denver'],
 [['Mick Mixon', 'Mick Mixon', 'Mick Mixon'], 'Dave Logan'],
 [['Dave Logan', 'Dave Logan', 'Dave Logan'], 'Dave Logan'],
 [['Ed McCaffrey', 'Ed McCaffrey', 'Ed McCaffrey'], 'Dave Logan'],
 [['WBT', 'WBT (1110 AM)', 'WBT (1110 AM)'], 'KOA'],
 [['Mick Mixon', 'Mick Mixon', 'Mick Mixon'], 'Dave Logan'],
 [['KOA (850 AM) and KRFX (103.5 FM)',
   'KOA (850 AM) and KRFX (103.5 FM)',
   'KOA (850 AM) and KRFX (103.5 FM)'],
  'KOA'],
 [['WBT-FM (99.3 FM)', 'WBT (1110 AM)', 'WBT-FM'], 'KOA'],
 [['Jonathan Stewart', 'Jonathan Stewart', 'Stewart'], 'punt from both teams'],
 [['Brad Nortman', 'Brad Nortman', 'Nortman'], 'both teams'],
 [['28', '28', '28'], 'Carolina'],
 [['61', '61', '61'], 'Carolina'],
 [['33', '33', '33'], 'Carolina'],
 [['51', '51', '51'], '73'],
 [['Jonathan Stewart', 'Jonathan Stewart', 'Stewart'], 'Carolina'],
 [['11:28', '11:28', '11:28'], 'Carolina'],
 [['Jordan Norwood', 'Jordan Norwood', 'Norwood'],
  'Carolina got on track with a 9-play, 73-yard scoring drive.'],
 [['33', '33', '33'], '73-yard scoring drive.'],
 [['Jonathan Stewart', 'Jonathan Stewart', 'Stewart'], 'Carolina'],
 [['field goal', 'field goal', 'field goal'], 'Carolina'],
 [['Ted Ginn Jr.', 'Ted Ginn Jr', 'Ted Ginn Jr'], 'Ted Ginn Jr.'],
 [['Graham Gano', 'Graham Gano', 'Gano'],
  'Newton completed a 45-yard pass to Ted Ginn Jr.'],
 [['44', '44', '44'], 'The Panthers'],
 [['McManus', 'McManus', 'McManus'], 'Newton'],
 [['T. J. Ward', 'T. J. Ward', 'Ward'], 'The Panthers'],
 [['Ted Ginn Jr.', 'Ted Ginn Jr.', 'Ted Ginn Jr'], 'Ted Ginn Jr.'],
 [['the uprights', 'the uprights', 'uprights'], 'The Panthers'],
 [['T. J. Ward.', 'T. J. Ward', 'Ward'], 'Ted Ginn Jr.'],
 [['Trevathan', 'Trevathan', 'Trevathan'], 'The Panthers'],
 [['Ted Ginn Jr.', 'Ted Ginn Jr'], 'Ted Ginn Jr.'],
 [['26-yard line', '26'], 'Denver 35-yard line'],
 [['Graham Gano', 'Graham Gano', 'Gano'],
  'Newton completed a 45-yard pass to Ted Ginn Jr.'],
 [['Emmanuel Sanders', 'Emmanuel Sanders', 'Sanders'], 'Newton'],
 [['24', 'their own 24', '24'], '24-yard line'],
 [['Newton', 'Newton', 'Newton'], 'Miller'],
 [['Josh Norman', 'Josh Norman', 'Norman'], '4:51'],
 [['Anderson', 'Anderson', 'Anderson'], 'Miller'],
 [['Bennie Fowler', 'Bennie Fowler', 'Fowler'],
  '4:51 left in regulation, Carolina got the ball on their own 24-yard line'],
 [['Miller', 'Miller', 'Miller'], 'Miller'],
 [['wards', 'Ward', 'Ward'], 'Miller'],
 [['Newton', 'Newton', 'Newton'], 'Miller'],
 [['Josh Norman', 'Josh Norman', 'Norman'], '4:51'],
 [['3:08', '3:08', '3:08'], '4:51'],
 [['4:51', '4:51', '4:51'], '4:51'],
 [['Miller', 'Miller', 'Miller'], 'Miller'],
 [['wards', 'Ward', 'Ward'], 'Miller'],
 [['three', 'three', 'three'],
  '4:51 left in regulation, Carolina got the ball on their own 24-yard line'],
 [['Anderson', 'Anderson', 'Anderson'],
  '4:51 left in regulation, Carolina got the ball on their own 24-yard line'],
 [['five', '2½', '2'], 'five'],
 [['zero', 'zero', 'zero'], 'zero'],
 [['four', 'four', 'four'], 'four'],
 [['Thomas Davis', 'Thomas Davis', 'Davis'], 'Manning'],
 [['one', 'one', 'one'], '18 of 41 passes for 265'],
 [['one', 'one', 'one'], 'one interception and zero touchdowns'],
 [['zero', 'zero', 'zero'], 'zero'],
 [['Anderson', 'Anderson', 'Anderson'], 'Anderson'],
 [['Sanders', 'Sanders', 'Sanders'], 'Sanders'],
 [['Thomas Davis', 'Thomas Davis', 'Davis'], 'Manning'],
 [['Sanders', 'Sanders', 'Sanders'], 'Sanders'],
 [['Anderson', 'Anderson', 'Anderson'], 'Anderson'],
 [['all four', 'all four', 'four'], 'all four'],
 [['one', 'one', 'one'], '18 of 41'],
 [['four', 'four', 'four'],
  'five solo), 2½ sacks, and two forced fumbles. Ware had five total tackles and two sacks.'],
 [['194', '194', '194'], '315 to 194'],
 [['11', '11', '11'], '21 to 11'],
 [['Baltimore Ravens', 'the Baltimore Ravens', 'Ravens'], 'Baltimore Ravens'],
 [['Jordan Norwood', 'Jordan Norwood', 'Norwood'], 'Jordan Norwood'],
 [['Manning', 'Manning', 'Manning'], 'Manning and Newton'],
 [['194', '194', '194'], '315 to 194'],
 [['11', '11', '11'], '21 to 11'],
 [['Chicago Bears', 'the Chicago Bears', 'Bears'], 'Kony Ealy'],
 [['Broncos', 'Denver', 'Denver'], 'Denver'],
 [['21', '21', '21'], '21 to 11'],
 [['11', '11', '11'], '21 to 11'],
 [['The Broncos', 'Denver', 'Denver'], 'Denver'],
 [['Chicago Bears', 'the Chicago Bears', 'Bears'], 'Chicago Bears'],
 [['two', 'two', 'two'], 'seven'],
 [['100', 'over 100', '100'], 'over 100'],
 [['Krasiński Palace Garden',
   'Krasiński Palace Garden',
   'Krasiński Palace Garden'],
  'Krasiński Palace Garden is a popular strolling destination for the Varsovians. The Monument of the Warsaw Ghetto Uprising'],
 [['The Saxon Garden', 'Saxon Garden', 'The Saxon Garden'],
  'The Saxon Garden'],
 [['east end', 'east', 'east'], 'east'],
 [['Łazienki', 'Łazienki Park', 'Łazienki Park'], 'The Łazienki Park'],
 [['Economist Intelligence Unit',
   'Economist Intelligence Unit',
   'Economist Intelligence Unit'],
  'Economist Intelligence Unit'],
 [['2012', '2012', '2012'], '2012'],
 [['Stock', 'Warsaw Stock Exchange', 'Warsaw Stock Exchange'],
  '32nd most liveable city in the world.'],
 [['Frontex', 'Frontex', 'Frontex'], '32nd most liveable city in the world.'],
 [['1313', '1313', '1313'], '1313'],
 [['Kraków', 'Kraków', 'Kraków'], 'Kraków'],
 [['1596', '1596', '1596'], '1313'],
 [['King Sigismund III Vasa',
   'King Sigismund III Vasa',
   'King Sigismund III Vasa'],
  'King Sigismund III Vasa'],
 [['survived many wars, conflicts and invasions',
   'it has survived many wars, conflicts and invasions',
   'because it has survived many wars, conflicts and invasions throughout its long history'],
  'because it has survived many wars, conflicts and invasions throughout its long history.'],
 [['Warszawa', 'Warszawa', 'Warszawa'], 'Warszawa'],
 [['belonging to Warsz', 'belonging to Warsz', 'belonging to Warsz'],
  'belonging'],
 [['12th/13th-century nobleman',
   'shortened form of the masculine name of Slavic origin Warcisław',
   'nobleman'],
  'Warcisław'],
 [['a village', 'village'],
  "Warsaw's name in the Polish language is Warszawa"],
 [['miasto stołeczne Warszawa',
   'miasto stołeczne Warszawa',
   'miasto stołeczne Warszawa'],
  'Warszawa'],
 [['about 300', '300', '300'], '300'],
 [['325', '325', '325'], '523'],
 [['Vistula River', 'Vistula', 'Vistula'], 'Vistula River'],
 [['452.8 ft', '115.7 metres', '115.7 metres'], '115.7 metres'],
 [['at the right bank of the Vistula',
   'at the right bank of the Vistula, by the eastern border of Warsaw',
   'by the eastern border'],
  '75.6 metres'],
 [['two', 'two', 'two'], 'two'],
 [['Vistula Valley', 'geomorphologic', 'Vistula Valley'], 'Vistula Valley'],
 [['moraine', 'moraine', 'moraine'], 'moraine plateau and the Vistula Valley'],
 [['Vistula River', 'Vistula River', 'Vistula River'], 'The Vistula River'],
 [['Warsaw Escarpment', 'Warsaw Escarpment', 'Warsaw Escarpment'],
  'Vistula Valley'],
 [['moraine', 'moraine', 'moraine'], 'The plain moraine'],
 [['former flooded terraces',
   'former flooded terraces',
   'peat swamps or small ponds'],
  'a few natural and artificial ponds and also groups of clay pits'],
 [['valleys', 'valleys and ground depressions', 'valleys'],
  'The plain moraine plateau has only a few natural and artificial ponds and also groups of clay pits'],
 [['plain Vistula terraces', 'highest terrace', 'highest terrace'],
  'plain moraine plateau'],
 [['pine', 'pine', 'pine'], 'Vistula terraces is asymmetrical.'],
 [['Gothic', 'Gothic', 'Gothic'], 'Gothic'],
 [['14th century', '14th century', '14th century'], '14th century'],
 [['Masovian gothic', 'Gothic architecture', 'Masovian gothic'], 'Gothic'],
 [['Renaissance', 'Renaissance', 'Renaissance'], 'Renaissance'],
 [['mannerist architecture', 'mannerist architecture', 'mannerist'],
  'Curia Maior'],
 [['17th century', '17th century', '17th century.'], '17th century'],
 [['1688–1692', '1677–1683', '1688–1692'], '1688–1692'],
 [['rococo', 'rococo', 'rococo'], 'rococo'],
 [['neoclassical architecture',
   'neoclassical architecture',
   'neoclassical architecture'],
  'neoclassical architecture'],
 [['1775–1795', '1696', '1775–1795'], '1775–1795'],
 [['green', 'green', 'green'], 'green'],
 [['New Orangery', 'New Orangery', 'New Orangery'], 'New Orangery'],
 [['location of Warsaw', 'location of Warsaw', 'location'], 'due'],
 [['within the borders of Warsaw', 'Warsaw', 'within the borders of Warsaw'],
  'Warsaw'],
 [['Masovian Primeval Forest',
   'Masovian Primeval Forest',
   'Masovian Primeval Forest'],
  'flora of the city may be considered very rich in species.'],
 [['Kabaty', 'Kabaty Forest', 'Kabaty'], 'several big floral regions'],
 [['two', 'two', 'two'], 'very rich in species.'],
 [['1,300,000', '1,300,000', '1,300,000'], 'c. 1,300,000'],
 [['420,000', '420,000', '420,000'], '420,000'],
 [['1951', '1951', '1951'], '1939'],
 [['as better', 'thought of themselves as better', 'better'], '420,000'],
 [['residency registration',
   'residency registration',
   'residency registration'],
  '420,000'],
 [['a commune', 'commune', 'commune'], 'a commune'],
 [['counties or powiats', 'counties or powiats', 'counties or powiats'],
  'counties or powiats'],
 [['Kraków', 'Kraków'], 'Lublin, Kraków, Gdańsk, Poznań'],
 [['Śródmieście', 'Śródmieście', 'Śródmieście'], 'Śródmieście'],
 [['304,016', '304,016'], '304,016'],
 [['emerging market', 'emerging market', 'emerging market'],
  'emerging market'],
 [['12%', '12%', '12%'], '12%'],
 [['191.766 billion PLN', '191.766 billion PLN', '191.766 billion PLN'],
  '191.766 billion'],
 [['William the Conqueror', 'William the Conqueror', 'William the Conqueror'],
  'The Norman dynasty'],
 [['Richard I', 'Richard I', 'Richard I'], 'The Norman dynasty'],
 [['Catholic', 'Catholic orthodoxy', 'Catholic'], 'Catholic orthodoxy'],
 [['911', '911', '911'], '10th century'],
 [['King Charles III', 'King Charles III', 'King Charles III'],
  'Norse war bands into the rivers of France evolved into more permanent encampments that included local women and personal property. The Duchy of Normandy'],
 [['Seine', 'Epte', 'Seine'], 'France'],
 [['999', 'In 999', '999'], 'Southern Italy'],
 [['Archangel Michael', 'the Archangel Michael', 'Archangel Michael'],
  'Normans'],
 [['Monte Gargano', 'at Monte Gargano', 'Monte Gargano'], 'Normans'],
 [['Drogo', 'Drogo', 'Drogo'], 'Tancred of Hauteville'],
 [['William Iron Arm', 'William Iron Arm', 'William Iron Arm'],
  'Tancred of Hauteville'],
 [['Kitab Rudjdjar', 'Kitab Rudjdjar', 'Kitab Rudjdjar'], 'Normans'],
 [['The Book of Roger', 'The Book of Roger', 'The Book of Roger'], 'Normans'],
 [['meritocratic', 'meritocratic', 'meritocratic'],
  'the Byzantines, Arabs, and Lombards'],
 [['Turkish forces', 'Turkish forces', 'Turkish forces'], 'Some'],
 [['Robert Guiscard', 'Robert Guiscard', 'Robert Guiscard'],
  'Robert Guiscard'],
 [['1082', 'February 1082', 'February 1082'], 'February 1082'],
 [['30,000', '30,000', '30,000'], '30,000'],
 [['1169', '1169', '1169'], '1169'],
 [['Ireland', 'Ireland', 'Ireland'], 'Irish'],
 [['Irish', 'Irish', 'Irish'], 'Irish culture'],
 [['Sybilla of Normandy', 'Sybilla of Normandy', 'Sybilla'],
  'Sybilla of Normandy'],
 [['Norman', 'Norman', 'Norman'], 'Normans'],
 [['Conrad of Montferrat', 'Conrad of Montferrat', 'Conrad of Montferrat'],
  'Conrad'],
 [['silver', 'silver', 'silver'],
  'Limassol at the same time, in particular Guy de Lusignan.'],
 [['Guy de Lusignan', 'Guy de Lusignan', 'Guy de Lusignan'],
  'Guy de Lusignan. All declared their support for Richard provided that he support Guy against his rival Conrad of Montferrat.'],
 [['early 11th century', '11th century', 'in the early 11th century'],
  '11th century'],
 [['dukes', 'the dukes', 'dukes'], 'the dukes'],
 [['mosaics', 'mosaics', 'mosaics'], 'stonework or metalwork'],
 [['1875', '1875', '1875'], '1875'],
 [['Austrian Polytechnic', 'Austrian Polytechnic', 'Austrian Polytechnic'],
  'Austrian Polytechnic'],
 [['1879', '1879', '1879'], '1879'],
 [['gambling', 'gambling', 'gambling'],
  'Professor Poeschl over the Gramme dynamo'],
 [['no', 'not', 'He never graduated'], '1875'],
 [['Graz, Austria', 'in Graz, Austria', 'Graz, Austria'], 'Graz, Austria'],
 [['1875', 'In 1875', '1875'], '1875'],
 [['1879', 'in 1879', '1879'], '1879'],
 [['gambled', 'gambling', 'gambling'], 'gambling'],
 [['Tesla would be killed through overwork',
   'killed through overwork',
   'would be killed through overwork'],
  'killed through overwork'],
 [['fifty thousand dollars',
   'fifty thousand dollars',
   'fifty thousand dollars'],
  'Edison Company'],
 [['$10 a week raise',
   'a US$10 a week raise',
   "a US$10 a week raise over Tesla's US$18 per week salary"],
  'Edison Company'],
 [['months', 'months', 'months'], 'Edison Company'],
 [['fifty thousand dollars',
   'fifty thousand dollars',
   'fifty thousand dollars'],
  "Edison Company's direct current generators."],
 [['American humor.',
   "Tesla, you don't understand our American humor",
   "you don't understand our American humor"],
  '1885'],
 [['Thomas Edison and George Westinghouse',
   'Thomas Edison and George Westinghouse',
   'Thomas Edison and George Westinghouse'],
  'Thomas Edison and George Westinghouse'],
 [['lighting systems',
   'electrical distribution',
   'a competition between rival lighting systems'],
  'lighting systems'],
 [['AC development', 'AC development', 'AC development'],
  "Tesla's demonstration of his induction motor"],
 [['General Electric', 'General Electric', 'General Electric'],
  'Westinghouse'],
 [['AC', 'AC', 'AC'], 'AC" side of the so-called "War of Currents'],
 [['Thomas Edison', 'Thomas Edison', 'Thomas Edison'],
  'Thomas Edison and George Westinghouse'],
 [['1888', '1888', '1888'], '1888'],
 [['financial strain',
   'financial strain',
   'the financial strain of buying up patents and hiring the engineers needed to build'],
  'Westinghouse'],
 [['General Electric', 'General Electric', 'General Electric'],
  'George Westinghouse'],
 [['Richard Dean Adams', 'Richard Dean Adams', 'Richard Dean Adams'],
  'Richard Dean Adams'],
 [['Niagara Falls', 'Niagara Falls', 'Niagara Falls'],
  'Niagara Falls Cataract Construction Company'],
 [['Westinghouse Electric', 'Westinghouse Electric', 'Westinghouse Electric'],
  'Cataract Construction Company'],
 [['General Electric', 'General Electric', 'Westinghouse Electric'],
  'Cataract Construction Company'],
 [['a two-phased system', 'a two-phased system', 'two-phased system'],
  'what system'],
 [['Richard Dean Adams', 'Richard Dean Adams'], 'Richard Dean Adams'],
 [['1893', 'In 1893', '1893'], '1893'],
 [['two-phased system', 'a two-phased system', 'two-phased system'],
  'what system'],
 [['most reliable',
   'most reliable',
   'would be the most reliable and that there was a Westinghouse system to light incandescent bulbs using two-phase alternating current'],
  '1893'],
 [['1896', '1896', '1896'], '1896'],
 [['$216,000', '$216,000', 'a lump sum payment of $216,000'], '$200,000'],
 [['$2.50 per AC horsepower royalty',
   '$2.50 per AC horsepower',
   '$2.50 per AC horsepower'],
  '$200,000'],
 [['$200,000', 'estimated $200,000', 'an estimated $200,000'], '$200,000'],
 [['J. P. Morgan', 'J. P. Morgan', 'financier J. P. Morgan'], 'J. P. Morgan'],
 [['an estimated $200,000',
   'estimated $200,000',
   'an estimated $200,000 in licenses and royalties'],
  '$200,000'],
 [['$216,000', '$216,000', 'a lump sum payment of $216,000'], '$200,000'],
 [['he had noticed damaged film in his laboratory in previous experiments',
   'he had noticed damaged film in his laboratory',
   'after he had noticed damaged film in his laboratory in previous experiments'],
  'he had noticed damaged film in his laboratory'],
 [['5th Avenue laboratory fire of March 1895', 'fire', 'fire'], '$50,000'],
 [['December 1895', 'December 1895', 'December 1895'], '1894'],
 [['the metal locking screw on the camera lens',
   'the metal locking screw',
   'the metal locking screw on the camera lens'],
  '1894'],
 [['1894', '1894', '1894'], '1894'],
 [['X-Rays', 'x-rays', '"Roentgen rays" or "X-Rays"'], 'invisible" kinds'],
 [['lost in the 5th Avenue laboratory fire of March 1895',
   'was lost',
   'was lost in the 5th Avenue laboratory fire of March 1895'],
  "Crookes tubes, a cold cathode electrical discharge tube. Soon after, much of Tesla's early research"],
 [['X-ray image', 'X-ray', 'X-ray image'], 'Roentgen rays" or "X-Rays").'],
 [['Mark Twain', 'Mark Twain', 'Mark Twain'], '1894'],
 [['1900', '1900', '1900'], '1900'],
 [['Marconi', 'Guglielmo Marconi', 'Guglielmo Marconi'], 'Guglielmo Marconi'],
 [['1901', '1901', '1901'], '1901'],
 [['1943', '1943', '1943'], '1900'],
 [['Supreme Court of the United States',
   'Supreme Court of the United States',
   'Supreme Court of the United States'],
  '1900'],
 [['communications from another planet',
   'communications from another planet',
   'communications from another planet'],
  'another planet'],
 [['Mars', 'Mars', 'Mars'], 'Mars'],
 [["Collier's Weekly", "Collier's Weekly", "Collier's Weekly"],
  "Collier's Weekly"],
 [["intercepted Marconi's European experiments",
   "he may have intercepted Marconi's European experiments",
   "he may have intercepted Marconi's European experiments in July 1899"],
  'another planet'],
 [['July 1899', 'July 1899', 'July 1899'], '1899'],
 [['over 50 letters', 'over 50 letters', '50 letters'], 'Marconi'],
 [['187 feet', '187 feet', '187 feet (57 m)'], '1901'],
 [['steam', 'steam', 'steam-powered'], 'steam'],
 [['Houston Street lab', 'his Houston Street lab', 'his Houston Street lab'],
  'Houston Street lab'],
 [['the machine oscillated at the resonance frequency of his own building',
   'the danger',
   'the machine oscillated at the resonance frequency of his own building'],
  'Tesla invented a steam-powered mechanical oscillator'],
 [['World Today', 'World Today', 'World Today'],
  'Tesla invented a steam-powered mechanical oscillator'],
 [['eventually split the earth in two',
   'destroy civilization',
   "he could set the earth's crust into such a state of vibration that it would rise and fall hundreds of feet and practically destroy civilization"],
  'Tesla invented a steam-powered mechanical oscillator'],
 [['feed the pigeons', 'feed the pigeons', 'feed the pigeons'],
  'make his regular commute to the cathedral and the library'],
 [['a doctor', 'a doctor', 'a doctor'],
  '1937, after midnight one night, Tesla left the Hotel New Yorker'],
 [['broken', 'broken', 'broken'], '1937'],
 [['early 1938', 'In early 1938', 'early 1938'], '1937'],
 [['the fall of 1937', 'In the fall of 1937', 'fall of 1937'], '1937'],
 [['eight', 'eight', 'eight'], 'eight'],
 [['visions', 'blinding flashes of light', 'visions'],
  'blinding flashes of light'],
 [['picture thinking', 'picture thinking'], 'photographic memory'],
 [['blinding flashes of light', 'flashbacks', 'blinding flashes of light'],
  'blinding flashes of light'],
 [['photographic memory', 'photographic', 'photographic'],
  'photographic memory'],
 [['more than 48 hours', '48 hours', '48 hours'], 'billiards'],
 [['Graz', 'Graz', 'Graz'], 'Graz'],
 [['Kenneth Swezey', 'Kenneth Swezey', 'Kenneth Swezey,'], 'Graz'],
 [['A Turing machine', 'A Turing machine', 'Turing machine'],
  'A Turing machine'],
 [['an algorithm', 'an algorithm', 'an algorithm'],
  'Turing machine is a mathematical model of a general computing machine.'],
 [['the Turing machine', 'the Turing machine', 'Turing machine'],
  'A Turing machine'],
 [['symbols', 'symbols', 'symbols'], 'symbols'],
 [['A deterministic Turing machine',
   'deterministic',
   'deterministic Turing machine'],
  'deterministic Turing machine'],
 [['rules', 'rules', 'a fixed set of rules to determine its future actions'],
  'fixed set of rules'],
 [['A probabilistic Turing machine',
   'probabilistic',
   'probabilistic Turing machine'],
  'probabilistic Turing machine'],
 [['A non-deterministic Turing machine',
   'non-deterministic',
   'non-deterministic Turing machine'],
  'deterministic Turing machine'],
 [['randomized algorithms', 'randomized algorithms', 'randomized algorithms'],
  'probabilistic Turing machine'],
 [['state transitions',
   'the total number of state transitions, or steps',
   'total number of state transitions, or steps, the machine makes before it halts and outputs the answer'],
  'time required by a deterministic Turing machine M on input x'],
 [['difficulty', 'difficulty', 'difficulty'], 'time and space'],
 [['DTIME(f(n))', 'DTIME(f(n)).', 'DTIME(f(n))'], 'computational model'],
 [['time', 'time', 'time'], 'time and space'],
 [['the most efficient algorithm',
   'the most efficient algorithm',
   'the most efficient algorithm solving a given problem'],
  'the most efficient algorithm'],
 [['analysis of algorithms',
   'analysis of algorithms',
   'analysis of algorithms'],
  'computation time'],
 [['lower bounds', 'lower', 'lower bounds'], 'upper and lower bounds'],
 [['upper bound', 'upper and lower bounds', 'upper bound'],
  'computation time'],
 [['all possible algorithms',
   'all possible algorithms',
   'all possible algorithms'],
  'computation time'],
 [['NP-complete', 'NP-complete', 'NP-complete'],
  'X is in C and hard for C, then X is said to be complete for C. This means that X is the hardest problem in C.'],
 [['NP', 'P', 'P'],
  'X is in C and hard for C, then X is said to be complete for C. This means that X is the hardest problem in C.'],
 [['there is no known polynomial-time solution',
   'no known polynomial-time solution',
   'there is no known polynomial-time solution'],
  'X is said to be complete for C. This means that X is the hardest problem in C.'],
 [['NP', 'NP', 'NP'], 'X is said to be complete for C'],
 [['The integer factorization problem',
   'integer factorization',
   'integer factorization problem'],
  'The integer factorization problem'],
 [['k', 'k', 'k'], 'k'],
 [['modern cryptographic systems',
   'modern cryptographic systems',
   'RSA algorithm'],
  'RSA algorithm'],
 [['the general number field sieve', 'RSA', 'general number field sieve'],
  'No efficient integer factorization algorithm is known, and this fact forms the basis of several modern cryptographic systems, such as the RSA algorithm.'],
 [['intractable problems',
   'intractable problems',
   'intractable',
   'intractable'],
  'intractable problems'],
 [['exponential-time algorithms',
   'exponential-time',
   'exponential-time algorithms',
   'exponential-time algorithms'],
  'exponential-time algorithms'],
 [['NP-complete problems', 'NP-complete', 'NP-complete', 'NP-complete'],
  'NP-complete'],
 [['particular skills', 'educational', 'particular skills'], 'skills'],
 [['self-study and problem solving',
   'classroom',
   'in self-study and problem solving'],
  'Teaching using pedagogy'],
 [['encourage', 'encourage', 'encourage'],
  'differentiated instruction as well as supervision to meet the needs of all students in the classroom. Pedagogy can be thought of in two manners.'],
 [['deflate', 'deflate', 'deflate'],
  'differentiated instruction as well as supervision to meet the needs of all students in the classroom. Pedagogy can be thought of in two manners.'],
 [['a coach', 'coach', 'coach'], 'Teaching using pedagogy'],
 [['England', 'England', 'England'], 'England'],
 [['priests, religious leaders, and case workers as well as teachers',
   'priests, religious leaders, and case workers as well as teachers',
   'priests, religious leaders, and case workers as well as teachers'],
  'priests, religious leaders, and case workers'],
 [['2,869', '2,869', '2,869'], '0.3%'],
 [['The AAUW study', 'AAUW', 'AAUW study'], 'England']]